# Tutorial 6 — Multi-Class Perceptron

**Author:** [Vito Dichio](https://sites.google.com/view/vito-dichio/home),
Fellow in AI, ENS–PSL, Paris — ✉️ vito.dichio@psl.eu

**Course:** *Machine Learning: Theory and Applications*
Master in Cognitive Sciences, ENS–PSL — Fall 2025/26 (Lecturer: Simona Cocco)

**Format:** Practical Session (TD)

#### Bibliography:

[1] Cocco et al., *From Statistical Physics to Data-Driven Modelling: with Applications to Quantitative Biology*, Oxford University Press (2022)



# Part A: Critical Capacity of the Perceptron

### Theoretical Background

In Tutorial 5, you applied the perceptron to MNIST digits (0 vs 1), which have structure and patterns. But what happens when we try to memorize **completely random associations** between inputs and outputs?

We consider $M$ **binary random patterns:** both inputs $x^{(m)} \in \{0, 1\}^L$ and labels $y^{(m)} \in \{0, 1\}$ are drawn uniformly at random, with no correlation. Here, $L$ is the dimension of the input space (number of features) and $M$ is the number of patterns to classify.

Consider a perceptron learning problem with **loading parameter** $\alpha = M/L$. For random binary patterns in the double infinite size limit $M, L \to \infty$ with fixed ratio $\alpha = M/L$, the **critical capacity** is $\alpha_c \approx 2$. This means:
- **$\alpha < \alpha_c$**: Random patterns are typically linearly separable $\Rightarrow$ perceptron converges
- **$\alpha > \alpha_c$**: Patterns typically NOT linearly separable $\Rightarrow$ perceptron fails to converge

For finite $L$ the transition is not infinitely sharp but occurs over a range around $\alpha_c \approx 2$. The transition becomes sharper as $L$ increases (see [1] Sec. 7.2.2, Figure 7.5 for illustration of finite-size effects).

**Your task:** Empirically verify this theorem by training perceptrons on random patterns for various $\alpha$ values and observing the convergence rate transition at $\alpha_c \approx 2$.

### 🎯 Question A.1: Generate Random Patterns

Implement a function called `generate_random_patterns` to generate random binary patterns and labels. The function should take as input the number of patterns `M`, the dimension `L`, and a random seed for reproducibility.

**Requirements:**
- Use `np.random.default_rng(seed)` for the random number generator
- Both patterns and labels should be uniformly random in $\{0, 1\}$
- Ensure not all labels are identical (redraw if needed, max 10 attempts)
- Each call with the same seed should produce identical results

Test your function for reproducibility by calling it twice with the same seed and checking that the outputs match. Use as `M`, `L` and `seed` the day, month and year of your **oral exam date** (e.g., for December 18, 2025 use `M=18`, `L=12`, `seed=2025`).

### 🎯 Question A.2: Test Capacity for Single Configuration

Implement a function `test_capacity` that tests perceptron capacity for a range of $\alpha$ values with fixed $L$. The function should take as input:
- `alpha_values`: array of $\alpha$ values to test
- `L`: dimension of input patterns
- `n_trials`: number of random trials per $\alpha$
- `max_updates`: maximum perceptron updates per trial

**Algorithm for each $\alpha$:**
1. Compute the number of patterns given `alpha` and `L`
2. For each trial:
   - Generate random patterns **using as seed the trial index** for reproducibility
   - Train perceptron using the perceptron learning algorithm with a maximum of `max_updates` updates
   - Predict and compute **training error**
   - Count as "converged" if training error is $\approx 0$  up to a difference of `1e-6` (use `np.isclose`)
3. Compute convergence rate: `converged_count / n_trials`
4. Compute mean training error across all trials

The function should finally return a DataFrame with columns: `alpha`, `convergence-rate`, `training error` as output.

**Test your function:**
```python
# Quick test with small parameters
alpha_test = np.linspace(0.25, 4.0, 16)
df_test = test_capacity(alpha_test, L=10, n_trials=10, max_updates=1000)

print(df_test)
# Should show high convergence rate for α < 2, lower for α > 2
```

💡 **Hint:** You may want to reuse the functions  `perceptron_signed`, `linear_classifier_predict` and `compute_error_rate` for an efficient implementation of the perceptron training and evaluation.


### 🎯 Question A.3: Run Capacity Experiments

Run systematic experiments to verify the critical capacity $\alpha_c \approx 2$ and investigate how dimension $L$ and training time affect the transition.

#### (i) Run experiments

**Experimental parameters:**
- `alpha_values = np.linspace(0.5, 3.0, 11)` - Range of loading parameters
- `n_trials = 100` - Number of independent trials per configuration
- Test all 4 combinations of:
  - `L = 10, 50` - Pattern dimensions
  - `max_updates = 1000, 5000` - Training budget

**Task:** For each of the 4 configurations, run `test_capacity` and store results. You should obtain 4 DataFrames, one per combination of (`L`, `max_updates`).


#### (ii) Plot the results

Create **two plots** showing all 4 experimental configurations: in the first plot, show convergence rate vs $\alpha$; in the second plot, show mean training error vs $\alpha$.

Mark the theoretical critical capacity $\alpha_c \approx 2$ with a vertical dashed black line in both plots. Use distinct colors or markers for each configuration.


#### (iii) Interpret the results

Comment on your findings from the experiments and plots. Address the following points:

1. **Critical capacity identification:** At what $\alpha$ value does the convergence rate drop most sharply for each configuration? How close are these values to the theoretical prediction $\alpha_c = 2$?

2. **Finite-size effects:** Compare the transition width for $L=10$ vs $L=50$. Does the transition become sharper for larger $L$, as predicted by theory? Cf Figure 7.5 in [1].

3. **Role of training budget:** Does increasing `max_updates` from 1000 to 5000 change the results? Specifically, does it affect the convergence rate near the critical region? If so, in which direction and why?

4. **Error behavior:** Examine the training error plots:
   - How does mean training error evolve as $\alpha$ increases?
   - For $\alpha > 2$, why is the error non-zero even though the perceptron runs for many updates?
   - Compare error curves for different `max_updates`: does training longer always help?

# Part B: Multi-Class Classification with One-vs-One Perceptrons

## Introduction

In Tutorial 5, you trained a perceptron to distinguish between two MNIST digits (0 vs 1). But what if we want to classify all 10 digits (0-9)? This requires extending binary classification to **multi-class classification**.

The strategy we will use is **One-vs-One (OVO)**, where we train a separate binary classifier for each pair of classes. During prediction, each classifier votes for one of its two classes, and the class with the most votes wins. In this part, you will implement the **One-vs-One (OVO)** strategy and analyze its performance on the full MNIST dataset. The entire dataset contains 60,000 training samples and 10,000 test samples of handwritten digits (0-9) and can be loaded using the`load_mnist_data` function from Tutorial 4.

### 🎯 Question B.1: Preliminaries

Before implementing the OVO system, we need to prepare our data and ensure the perceptron can handle arbitrary class labels.

#### (i) Load MNIST Data

Load the full MNIST dataset (all 10 digits) and flatten the images. You should obtain:
- `x`: shape `(70000, 784)` - flattened training images
- `y`: shape `(70000,)` - training labels (0-9)

💡 **Hint:** You may use the `load_mnist_data` and `flatten_images` functions from Tutorial 4.

#### (ii) Split MNIST into training and test sets

After implementing, apply it to the MNIST images and then split it into training and test sets (60,000 train, 10,000 test).

💡 **Hint:** You can use the `split_train_test` function from Tutorial 4.

#### (iii) Make perceptron label-agnostic

The `perceptron_signed` and `linear_classifier_predict` functions from Tutorial 4 assume labels are exactly 0 and 1 (or -1 and +1). For OVO, we need to classify any pair of digits (e.g., 1 vs 7).

**Task:** Modify the perceptron function and prediction function to handle arbitrary binary labels by adding `pos_label` and `neg_label` parameters.

**For `perceptron_signed`:**
- Add a `pos_label` parameter (default: 1)
- Before creating signed patterns, map the input labels: `pos_label → +1`, other label → `-1`. Use `np.where` for this mapping.

**For `linear_classifier_predict`:**
- Add `pos_label` (default: 1) and `neg_label` (default: 0) parameters
- If score > theta: predict `pos_label`, otherwise predict `neg_label`

With these modifications, you can train on any digit pair by specifying which digit is the "positive" class. For example, `perceptron_signed(X, y, pos_label=1)` will treat 1 as +1 and the other digit (e.g., 7) as -1 internally, while still returning predictions in the original labels (1 and 7).

**Verify your implementation:**

```python
pos_label = 1
neg_label = 7

mask = (y_train == pos_label) | (y_train == neg_label)
x_pair = x_train[mask]
y_pair = y_train[mask]

# Train perceptron treating 8 as positive class
result = perceptron_signed(x_pair, y_pair, pos_label=pos_label, max_updates=2000)

# Predict using consistent label mapping
y_pred = linear_classifier_predict(x_pair, result['J'], theta=0,  pos_label=pos_label, neg_label=neg_label)

# Compute error
error = compute_error_rate(y_pair, y_pred, verbose=True)
print(f"Training error for {pos_label} vs {neg_label}: {error*100:.2f}%")
```

### 🎯 Question B.2: Train One-vs-One Perceptrons

The **One-vs-One (OVO)** strategy decomposes multi-class classification into multiple binary problems. For K classes, we train all possible pairwise classifiers:

$$\text{Number of classifiers} = \binom{K}{2} = \frac{K(K-1)}{2}$$

For MNIST with K=10 digits, this means training **45 binary perceptrons**:
- (0,1), (0,2), ..., (0,9): 9 classifiers involving digit 0
- (1,2), (1,3), ..., (1,9): 8 classifiers involving digit 1
- ...
- (8,9): 1 classifier for the last pair

Each binary classifier learns to distinguish between two specific digits using only the training samples from those two classes. For example, the (1,7) classifier is trained only on images of 1s and 7s, ignoring all other digits.

#### Implementation

Implement a function `train_ovo_perceptron` that takes as input the training data `X_train`, `y_train`, and trains an OVO perceptron system (45 binary classifiers) by calling your modified `perceptron_signed` function for each pair of classes. Use `max_updates=2000` for each binary classifier.

**Algorithm:**
1. Extract all unique class labels (digits) from `y_train` (use `np.unique`)
2. Initialize a data structure to store all trained classifiers
3. For each pair of classes (i, j) where i < j:
   - Extract training samples belonging to class i or class j (see example in B.1.iii)
   - Train a binary perceptron on this subset using `perceptron_signed`
   - Store the resulting weights and label information
4. Return the collection of all trained classifiers


💡**Storage strategy:**

You need to store 45 trained classifiers in a way to save for each of them:
- The pair of classes it distinguishes and which of them is positive/negative
- The trained weight vector

Possible approaches include using dictionaries with tuple keys, lists of tuples, or any other structure you find appropriate. Think about how you will later retrieve and use these classifiers for prediction.

⚠️ **Computational note:** Training on the full dataset with `max_updates=2000` may take **3-5 minutes** on a standard CPU. Start with smaller values to develop your code and debug, then run the full training once you're confident your implementation is correct.

### 🎯 Question B.3: Predict with Majority Voting

#### Voting Mechanism

Once all binary classifiers are trained, we need a strategy to combine their predictions for multi-class classification. The **One-vs-One voting** works as follows:

**For each test sample:**
1. Each of the K(K-1)/2 binary classifiers predicts one of its two classes
2. Count votes: each class accumulates votes from all classifiers that involve it
3. Predict the class with the most votes (winner-takes-all)

**Example for a sample that might be "7":**
- Classifier (0,7) → votes for 7
- Classifier (1,7) → votes for 7
- Classifier (2,7) → votes for 7
- ...
- Classifier (7,9) → might vote for 9 (misclassification)
- **Result:** Class 7 receives 8 votes, class 9 receives 1 vote → predict 7

Note that for K=10 classes, each sample receives exactly 45 votes (one from each binary classifier). For a given sample, each class can receive at most K-1=9 votes (from the 9 classifiers that involve that class).

#### Implementation

Implement `predict_ovo(X, classifiers)` that returns predicted class labels using majority voting.

💡 **Pseudocode:**
```
N ← number of samples in X

Extract all unique digit labels from your stored classifiers → digits (sorted list)

K ← number of unique digits

Initialize vote matrix: votes (dimensions N x K) with zeros

FOR each stored binary classifier:
    Retrieve: weights, positive digit label, negative digit label

    y_pred ← linear_classifier_predict(X, weights, pos_label, neg_label)

    idx_pos ← index of positive digit in digits list
    idx_neg ← index of negative digit in digits list

    # Add votes (vectorized or loop over samples)
    FOR those sample indices n where y_pred[n] == positive digit:
        votes[n, idx_pos] += 1
    FOR those sample indices n where y_pred[n] == negative digit:
        votes[n, idx_neg] += 1

# Winner-takes-all: find digit with most votes for each sample
FOR each sample n:
    winning_index ← argmax of the row votes[n, :]
    predictions[n] ← digits[winning_index]

RETURN predictions
```

Run the `predict_ovo` function on both the **train** and the **test** set and compute the overall accuracy by comparing with true labels.

### 🎯 Question B.4: Confusion Matrix Analysis

#### (i) Visualize Results

A **confusion matrix** shows how often each true digit is predicted as each possible digit. The entry in row $i$, column $j$ shows how many times digit $i$ was predicted as digit $j$.

- **Diagonal entries**: correct predictions (true label = predicted label)
- **Off-diagonal entries**: confusions (true label ≠ predicted label)

**Task:** Implement `plot_confusion_matrix(y_true, y_pred, ...)` that:
- Computes the confusion matrix using `sklearn.metrics.confusion_matrix` (See [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html))
- Creates a heatmap visualization with:
  - Color intensity showing number of samples
  - Axis labels indicating true (rows) vs predicted (columns) digits

**Generate confusion matrices for both train and test sets.**

### (ii) Interpret the Results

Analyze the confusion matrices and address the following questions in your report:

**1. Overall Performance**

- **Best performers:** Which digits have the highest diagonal values (most correct predictions)? Is this consistent with Tutorial 4 results for digits 0 and 1?

- **Per-digit accuracy:** How would you compute accuracy for each individual digit from the confusion matrix? Which digits have the lowest accuracy and why might this be? What would you suggest to improve performance on these digits?

- **Generalization:** Compare train vs test confusion matrices. Are the error patterns similar? Does the model generalize well, or do you observe overfitting for certain digits?

**2. Confusion Patterns and Symmetry**

- **Major confusions:** Identify the largest off-diagonal entries (most common mistakes). Speculate on why this confusion occurs based on visual similarity

- **Asymmetric confusions:** Examine whether the confusion matrix is generally not symmetric.  Identify at least two clear asymmetric pairs where confusion[i,j] >> confusion[j,i] and explain why these asymmetries might occur.

---

## ➕ Optional (Bonus) Questions

### 🎯 B.5 The Role of the Bias Term

In Tutorial 5, the perceptron learned decision boundaries of the form $J \cdot x = 0$, which must pass through the origin. For many digit pairs, the optimal separating hyperplane needs to be shifted away from the origin. Adding a bias feature $x \rightarrow [x; 1]$ allows the perceptron to learn boundaries of the form $J \cdot x + b = 0$, where the weight corresponding to the bias feature acts as $b$. See the discussion in [1]. Sec. 7.1.1, p. 138.

**Task:**  Implement a function `add_bias_feature(x)` that appends a constant feature (value = 1) to each sample. The function should take an array of shape `(N, features)` and return an array of shape `(N, features+1)` where the last column is all ones.

Add the bonus bias to the MNIST data before splitting into train/test sets. Then, retrain the OVO perceptron system and evaluate performance again. What changes do you observe in accuracy and confusion matrices compared to the no-bias case? Are the (mis-)classification patterns similar or different?

### 🎯 B.6 The Role of Training Budget

Investigate how increasing the `max_updates` parameter during training affects the OVO perceptron performance. Retrain the OVO system with a higher `max_updates=20000` and evaluate accuracy and confusion matrices again. Does training longer lead to significant improvements? Are certain digits more affected by the increased training budget than others? Warining: This may take **10-20** minutes to run!

---
